In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
pd.set_option("display.max_columns",500)
pd.set_option("display.max_rows",500)
import sklearn as sk
print(sk.__version__)


1.0.2


## Obtencion de la data

In [2]:
data0 = pd.read_csv('../data/modeling/01_raw.csv',encoding='ISO-8859-1')
data = data0

## Exploración de variables 

In [3]:
p25=np.percentile(data["MTOTRANSACCION"], 25)
p50=np.percentile(data["MTOTRANSACCION"], 50)
p75=np.percentile(data["MTOTRANSACCION"], 75)
p90=np.percentile(data["MTOTRANSACCION"], 90)
p95=np.percentile(data["MTOTRANSACCION"], 95)
print('p25 = ',p25,'p50 = ',p50,', p75 = ',p75,', p90 = ',p90 ,' ,p95 = ',p95)

p25 =  235979.95 p50 =  294928.635 , p75 =  396275.0 , p90 =  503433.99999999994  ,p95 =  651373.7949999986


In [4]:
data['FLG_MTO_P25']=0
data.loc[(data['MTOTRANSACCION']>=p25)& (data['MTOTRANSACCION']<p50),'FLG_MTO_P25']=1

In [5]:
data['FLG_MTO_P50']=0
data.loc[(data['MTOTRANSACCION']>=p50)& (data['MTOTRANSACCION']<p75),'FLG_MTO_P50']=1

In [6]:
data['FLG_MTO_P75']=0
data.loc[(data['MTOTRANSACCION']>=p75)& (data['MTOTRANSACCION']<p90),'FLG_MTO_P75']=1

In [7]:
data['FLG_MTO_P90']=0
data.loc[(data['MTOTRANSACCION']>=p90),'FLG_MTO_P90']=1

### Reemplazar los nulos 

In [8]:
data['FLG_AN_BENEFICIARIO'].isnull().sum()

0

In [9]:
data['FLG_AN_SOLICITANTE'].isnull().sum()

0

In [10]:
data['DESBANCA_BENEFICIARIO'].isnull().sum()

4

In [11]:
data['DESBANCA_BENEFICIARIO'].replace('[NULL]', np.nan,inplace=True)

In [12]:
data['TIPPER_BENEFICIARIO'].isnull().sum() 

4

In [13]:
data['TIPPER_BENEFICIARIO'].replace('[NULL]', np.nan,inplace=True)

In [14]:
data['TIPPER_SOLICITANTE'].replace('[NULL]', np.nan,inplace=True)

# Categorizar variables 

In [15]:
data['FECDIA2'] = pd.to_datetime(data['FECDIA'])

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14/02/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '23/06/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '28/02/2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '21/07/2020' in DD/MM/YYYY format. Provide format or specif

In [16]:
data['FEC_BEN'] = pd.to_datetime(data['FECONS_BENEFICIARIO'])

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '28/04/2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '24/05/1996' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18/04/2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '24/07/2006' in DD/MM/YYYY format. Provide format or specif

In [ ]:
data['FEC_SOL'] = pd.to_datetime(data['FECONS_SOL'])

## 1. Generación de Flag

In [17]:
data['FLG_AN']=0
data.loc[(data['FLG_AN_BENEFICIARIO']==1)& (data['FLG_AN_SOLICITANTE']==1),'FLG_AN']=1

In [18]:
data['TIPPER']=0
data.loc[(data['TIPPER_SOLICITANTE']=='P')& (data['TIPPER_BENEFICIARIO']=='P'),'TIPPER']=1

In [19]:
data["FLG_NUEVA_BEN"]=np.where(data["FECONS_BENEFICIARIO"].astype(str).str[6:10]==2020,1,0)

In [20]:
data['dif']   = data['FECDIA2'] - data['FEC_BEN']                              

In [21]:
data['dif2'] = data['dif'].dt.days 


In [22]:
data.loc[(data['TIPBANCA_BENEFICIARIO']=='E') & (data['FEC_BEN'].isnull()==True),:].shape

(19, 39)

In [23]:
data["FLG_NUEVA_SOL"]=np.where(data["FECONS_SOL"].astype(str).str[6:10]==2020,1,0)

In [24]:
data['FLG_NUEVA']=0

data.loc[(data['FLG_NUEVA_SOL']==1)& (data['FLG_NUEVA_BEN']== 1),'FLG_NUEVA']=1

In [25]:
data['FLG_BANCA']=0

In [26]:
data['PERSONA_SOL']=0
data.loc[(data['DESBANCA_SOLICITANTE']== 'BANCA PERSONAL                     '),'PERSONA_SOL']=1

In [27]:
data['EMPRESA_SOL']=0
data.loc[(data['DESBANCA_SOLICITANTE']== 'EMPRESA                            '),'EMPRESA_SOL']=1

In [28]:
data['PERSONA_BEN']=0
data.loc[(data['DESBANCA_BENEFICIARIO']== 'BANCA PERSONAL                     '),'PERSONA_BEN']=1

In [29]:
data['EMPRESA_BEN']=0
data.loc[(data['DESBANCA_BENEFICIARIO']== 'EMPRESA                            '),'EMPRESA_BEN']=1

In [30]:
data.loc[(data['DESBANCA_SOLICITANTE']== 'BANCA PERSONAL                     ')& (data['DESBANCA_BENEFICIARIO']== 'BANCA PERSONAL                     '),'FLG_BANCA']=1

In [31]:
data['FLG_ROS']=0

In [32]:
data.loc[(data['FLG_ROS_SOL']==1)& (data['FLG_ROS_BEN']==1),'FLG_ROS']=1

In [33]:
data['FLG_BANCA']=0
data.loc[(data['DESBANCA_SOLICITANTE']=='BANCA PERSONAL                     ','FLG_BANCA')]= 1

In [34]:
data["FLG_MTO"]=np.where(data["MTOTRANSACCION"]>500000,1,0)

In [35]:
data["FLG_NOREL"]=np.where(data["FLG_REL"]==1,0,1)

In [ ]:
#Escalar el monto 

In [36]:
data['MTOTRANSACCION_ES']=data['MTOTRANSACCION'].astype(float)

In [37]:
scale(data['MTOTRANSACCION_ES'], axis=0, with_mean=True, with_std=True, copy=True)

array([ 1.30268538,  0.02609956,  0.92697125, ..., -0.52651288,
       -0.25021771, -0.62048286])

In [38]:
data["TIPPER_BEN"]=np.where(data["TIPPER_BENEFICIARIO"]=='P',0,1)

In [39]:
data["TIPPER_SOL"]=np.where(data["TIPPER_SOLICITANTE"]=='P',0,1) 

In [40]:
p95=np.percentile(data["MTOTRANSACCION"], 95)

In [41]:
data["MTO_P95"]=np.where(data["MTOTRANSACCION"]>p95,p95,data["MTOTRANSACCION"])

In [42]:
data['MTOTRANSACCION_ES95']=data['MTO_P95'].astype(float)

In [43]:
data['MTO_P95'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3480 entries, 0 to 3479
Series name: MTO_P95
Non-Null Count  Dtype  
--------------  -----  
3480 non-null   float64
dtypes: float64(1)
memory usage: 27.3 KB


In [44]:
scale(data['MTOTRANSACCION_ES95'], axis=0, with_mean=True, with_std=True, copy=True)

array([ 1.98375298,  0.13721872,  1.44029636, ..., -0.66211478,
       -0.26246402, -0.79803889])

In [ ]:
data.to_csv('../data/modeling/02_preprocessed.csv', sep="|", index = False)